In [34]:
import numpy as np 
import matplotlib.pyplot as plt 
from math import pi 
from scipy import special

In [35]:
def f(x):
    a = 1 / 1000
    si, ci = special.sici(x / a)
    return si * np.exp(-x ** 2 / 2)

In [36]:
# generate data 
d = 2
N = 10**4

x_train = np.random.normal(0, 1, N*d).reshape((N, d))
y_train = f(x_train)

x_test = np.random.normal(0, 1, N*d).reshape((N, d))
y_test = f(x_test)

In [37]:
# normalize the data 

def normalize(x, y): 
    x_norm = (x - np.mean(x))/np.std(x, ddof=1)
    y_norm = (y - np.mean(y))/np.std(y, ddof=1)
    return x_norm, y_norm 

x_train, y_train = normalize(x_train, y_train)
x_test, y_test = normalize(x_test, y_test)

In [38]:
# minimization function 
def min_fun(x, y, w, lambda_): 
    I = np.identity(K)
    b = np.zeros(K*d).reshape((K, d))
    one = np.zeros(N*d).reshape((N, d))
    S = np.cos(x.dot(w.T) + one.dot(b.T))
    beta = np.linalg.inv(np.dot(S.T, S) + lambda_ * N * I).dot(S.T).dot(y)
    return beta, S

In [39]:
# Metropolis algorithm 1

def algorithm_1(x, y, M, K, d, lambda_, delta):

    w = np.zeros(K*d).reshape((K, d))
    
    beta, S = min_fun(x, y, w, lambda_)
    for i in range(M): 
        r_n = np.random.normal(0, 1, K*d).reshape((K, d))
        w_temp = w + delta * r_n
        beta_temp, S_temp = min_fun(x, y, w_temp, lambda_)
        for k in range(K): 
            r_u = np.random.uniform(0, 1)
            if np.linalg.norm(beta_temp[k], ord=1)/np.linalg.norm(beta[k], ord=1) > r_u: 
                w[k] = w_temp[k]
                beta[k] = beta_temp[k]
        beta, S = min_fun(x, y, w, lambda_)
        
    f = np.sum(S, axis=1)
        
    return beta, f

In [43]:
M = 10
K = 100 
lambda_ = 1 
delta = 2.4**2/d 

beta, f = algorithm_1(x_train, y_train,  M, K, d, lambda_, delta)

(10000,)

In [48]:
# Metropolis algorithm 2

def algorithm_2(x, y, M, K, d, lambda_, delta):
    
    w = np.zeros(K*d).reshape((K, d))
    S_w = np.zeros(d)
    S_c = np.zeros(d*d).reshape((d, d))
    C_w = np.zeros(d*d).reshape((d, d))
    C = np.ones(d*d).reshape((d, d))
    normal_mean = np.zeros(d)
    t0 = 10
    beta, S = min_fun(x, y, w, lambda_)
    
    for i in range(1, M+1): 
        r_n = np.random.multivariate_normal(normal_mean, C, K)
        w_temp = w + delta * r_n
        beta_temp, S_temp = min_fun(x_train, y_train, w_temp, lambda_)
        for k in range(K): 
            r_u = np.random.uniform(0, 1)
            if np.linalg.norm(beta_temp[k], ord=1)/np.linalg.norm(beta[k], ord=1) > r_u: 
                w[k] = w_temp[k]
                beta[k] = beta_temp[k]
            S_w = S_w + w[k]
            S_c = S_c + np.dot(w[k].T, w[k])
        w_temp_mean = S_w / (i*K)
        C_temp = S_c/(i*K) - np.dot(w_temp_mean.T, w_temp_mean)
        if i>t0: 
            C = C_temp
        beta, S = min_fun(x_train, y_train, w, lambda_)
        
    f = np.sum(S, axis=1)
        
    return beta, f

In [50]:
M = 10
K = 100 
lambda_ = 1 
delta = 2.4**2/d 

beta, f = algorithm_2(x_train, y_train,  M, K, d, lambda_, delta)

(10000,)